Пример, который обучается генерировать тексты похожие на тексты Ницце. 

In [2]:
#Python 2.7
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using Theano backend.


Скачиваем текст 

In [3]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600901


In [5]:


chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 59
nb sequences: 200287
Vectorization...


In [9]:
X.shape

(200287, 40, 59)

Keras довольно высокоуровневый интерфейс. 
Для построения модели достаточно просто описать какие слои вы хотите и в каком порядке. 

In [3]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Обучение работает очень долго. Имеет смысл дождаться хотя бы одной итерации и попытаться сгененировать текст. 


In [4]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

524288/600901 [=========================>....] - ETA: 0scorpus length: 600901
total chars: 59
nb sequences: 200287
Vectorization...
Build model...

--------------------------------------------------
Iteration 1


WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Epoch 1/1
200287/200287 [==============================] - 660s - loss: 2.0023   

----- diversity: 0.2
----- Generating with seed: "ch our existence and the maintenance of "
ch our existence and the maintenance of the constions of the constions of the constions, the defenged the stall the into the and the into the sense the in the constions of the intore the most the consige the distrution of the constion of the constion the intount the constion the intution of the constion of the intore the constions of the into the be the one the constion the into the something the stall the constions of the former the mo

----- diversity: 0.5
----- Generating with seed: "ch our existence and the maintenance of "
ch our existence and the maintenance of the highertides in a dution of the condition of the intelse of which who has he fore the deviry be the conscientian of the condigition of the prositian and has the is condition of the indection of the prouls of the power of the more into manifould in 

KeyboardInterrupt: 